In [1]:
def register_timelapse(video):
    """
    Registers with subpixel precision a 3D timelapse (T,Z,Y,X) using translations in Z, Y, X.
    The last frame is the reference frame, and it remains fixed.
    """
    from skimage import registration
    import scipy.ndimage as ndi

    new_video = video.copy()
    ref_frame = new_video[-1]
    for t in range(len(video) - 1, 0, -1):
        mov_frame = video[t - 1]
        shift, _, _ = registration.phase_cross_correlation(
            ref_frame,
            mov_frame,
            upsample_factor=4,
            normalization=None,
        )
        # replaces old ref_frame with shifted mov_frame
        ref_frame = ndi.shift(mov_frame, shift, order=1)
        new_video[t - 1] = ref_frame

    return new_video

In [2]:
def check(candidate):
    import numpy as np
    from skimage.feature import peak_local_max
    shape = (15, 15, 15)
    
    def _gaussian(mean):
        variance = 0.5

        support = np.stack(np.meshgrid(*[
            np.linspace(-s // 2, s // 2, s) + m
            for s, m in zip(shape, mean)
        ], indexing="ij"))

        density = np.exp(-np.linalg.norm(support, axis=0) / (2 * variance))
        return density

    video = np.stack([
        _gaussian((-3, 3, 0)),
        _gaussian((2, 0, -2)),
        _gaussian((0, 2, 0))
    ])
        
    ref_coord = peak_local_max(video[-1])
    res_coords = np.concatenate([
        peak_local_max(frame)
        for frame in candidate(video)
        # for frame in video  # sanity check making sure original video doesn't pass test
    ])

    max_dist = np.abs(res_coords - ref_coord)
    np.testing.assert_array_less(max_dist, 0.5)

In [3]:
check(register_timelapse)